In [8]:
import pandas as pd
import numpy as np
import os

In [9]:
## Read in files to get clean base dataframe
directory = "Data"
df_list = []

for root, dirs, files in os.walk(directory):
    if "INFOTABLE.tsv" in files and "OTHERMANAGER2.tsv" in files and "OTHERMANAGER.tsv" in files:
        intotable_path = os.path.join(root, "INFOTABLE.tsv")
        manager1_path = os.path.join(root, "OTHERMANAGER.tsv")
        manager2_path = os.path.join(root, "OTHERMANAGER2.tsv")
        manager1_df = pd.read_csv(manager1_path, sep="\t", usecols=["ACCESSION_NUMBER", "NAME"])
        manager2_df = pd.read_csv(manager2_path, sep="\t", usecols=["ACCESSION_NUMBER", "NAME"])

        manager_df = pd.concat([manager1_df, manager2_df], ignore_index=True)
        manager_df.rename(columns={'NAME':'FUND'}, inplace=True)

        infotable_df = pd.read_csv(intotable_path, sep = "\t", 
                                   usecols=["ACCESSION_NUMBER","NAMEOFISSUER","TITLEOFCLASS",
                                            "CUSIP", "VALUE", "SSHPRNAMT"])
        # Add date
        sub_dir = os.path.basename(root)
        datetime = sub_dir.split("-")[1].split("_")[0]
        infotable_df.insert(0, "Datetime", datetime)
        infotable_df["Datetime"] = pd.to_datetime(infotable_df["Datetime"])
        
        merged_df = pd.merge(infotable_df, manager_df, on="ACCESSION_NUMBER", how="inner") #fund name non-optional
        merged_df.drop(columns=["ACCESSION_NUMBER"], inplace=True)
        df_list.append(merged_df)

df_full = pd.concat(df_list, ignore_index=True)


## 

In [10]:
df_full

,Datetime,NAMEOFISSUER,TITLEOFCLASS,CUSIP,VALUE,SSHPRNAMT,FUND
0,2024-05-31,CREDICORP LTD,COM,G2519Y108,757896,5055,METLIFE INC
1,2024-05-31,PAGSEGURO DIGITAL LTD,COM CL A,G68707101,119700,9599,METLIFE INC
2,2024-05-31,XP INC,CL A,G98239109,48438,1858,METLIFE INC
3,2024-05-31,INTERCORP FINL SVCS INC,SHS,P5626F128,1336645,60895,METLIFE INC
4,2024-05-31,COPA HOLDINGS SA,CL A,P31076105,1680442,15807,METLIFE INC
...,...,...,...,...,...,...,...
35529354,2024-08-31,THERMO FISHER SCIENTIFIC INC,COM,883556102,207539,391,BANK VONTOBEL AG
35529355,2024-08-31,THERMO FISHER SCIENTIFIC INC,COM,883556102,207539,391,Vontobel Swiss Financial Advisers AG
35529356,2024-08-31,THERMO FISHER SCIENTIFIC INC,COM,883556102,207539,391,Bank Vontobel Europe AG
35529357,2024-08-31,THERMO FISHER SCIENTIFIC INC,COM,883556102,207539,391,Vontobel Asset Management Ltd


In [11]:
# Join with aum 
aum_df = pd.read_csv("Data/aum_df")
df_full = df_full.merge(aum_df, left_on='FUND', right_on="fund_name", how='left')

In [12]:
# Set multiindex fund-level and datetime level
df_full.drop("fund_name", inplace=True, axis=1)
df_full.set_index(["FUND", "Datetime"], inplace=True)

In [13]:
df_full

NAMEOFISSUER  \
FUND                                 Datetime                                   
METLIFE INC                          2024-05-31                 CREDICORP LTD   
                                     2024-05-31         PAGSEGURO DIGITAL LTD   
                                     2024-05-31                        XP INC   
                                     2024-05-31       INTERCORP FINL SVCS INC   
                                     2024-05-31              COPA HOLDINGS SA   
...                                                                       ...   
BANK VONTOBEL AG                     2024-08-31  THERMO FISHER SCIENTIFIC INC   
Vontobel Swiss Financial Advisers AG 2024-08-31  THERMO FISHER SCIENTIFIC INC   
Bank Vontobel Europe AG              2024-08-31  THERMO FISHER SCIENTIFIC INC   
Vontobel Asset Management Ltd        2024-08-31  THERMO FISHER SCIENTIFIC INC   
Vontobel Asset Management S.A.       2024-08-31  THERMO FISHER SCIENTIFIC INC   

                                                TITLEOFCLASS      CUSIP  \
FUND                                 Datetime                             
METLIFE INC                          2024-05-31          COM  G2519Y108   
                                     2024-05-31     COM CL A  G68707101   
                                     2024-05-31         CL A  G98239109   
                                     2024-05-31          SHS  P5626F128   
                                     2024-05-31         CL A  P31076105   
...                                                      ...        ...   
BANK VONTOBEL AG                     2024-08-31          COM  883556102   
Vontobel Swiss Financial Advisers AG 2024-08-31          COM  883556102   
Bank Vontobel Europe AG              2024-08-31          COM  883556102   
Vontobel Asset Management Ltd        2024-08-31          COM  883556102   
Vontobel Asset Management S.A.       2024-08-31          COM  883556102   

                                                   VALUE  SSHPRNAMT  \
FUND                                 Datetime                         
METLIFE INC                          2024-05-31   757896       5055   
                                     2024-05-31   119700       9599   
                                     2024-05-31    48438       1858   
                                     2024-05-31  1336645      60895   
                                     2024-05-31  1680442      15807   
...                                                  ...        ...   
BANK VONTOBEL AG                     2024-08-31   207539        391   
Vontobel Swiss Financial Advisers AG 2024-08-31   207539        391   
Bank Vontobel Europe AG              2024-08-31   207539        391   
Vontobel Asset Management Ltd        2024-08-31   207539        391   
Vontobel Asset Management S.A.       2024-08-31   207539        391   

                                                          aum    aum_date  
FUND                                 Datetime                              
METLIFE INC                          2024-05-31           NaN         NaN  
                                     2024-05-31           NaN         NaN  
                                     2024-05-31           NaN         NaN  
                                     2024-05-31           NaN         NaN  
                                     2024-05-31           NaN         NaN  
...                                                       ...         ...  
BANK VONTOBEL AG                     2024-08-31           NaN         NaN  
Vontobel Swiss Financial Advisers AG 2024-08-31  8.500000e+09  2023-03-29  
Bank Vontobel Europe AG              2024-08-31           NaN         NaN  
Vontobel Asset Management Ltd        2024-08-31           NaN         NaN  
Vontobel Asset Management S.A.       2024-08-31           NaN         NaN  

[35529359 rows x 7 columns]

In [14]:
# ADD RANK COLUMN
df_full['RANK'] = df_full.groupby(['FUND', 'Datetime'])['VALUE'].rank(ascending=False, method = 'dense')

In [15]:
# ADD columns
df_full['TotalHoldingsMarketValue'] = df_full.groupby(['FUND', 'Datetime'])['VALUE'].transform('sum')
df_full['Percentage'] = (df_full['VALUE'] / df_full['TotalHoldingsMarketValue']) * 100
# add change in percentage
df_full['Previous Percentage'] = df_full.groupby('FUND')['Percentage'].shift(1)
df_full['ChangeInPercentage'] = df_full['Previous Percentage'] - df_full['Percentage']

In [16]:
# Add AUM from results from test_aum


In [17]:
df_full

NAMEOFISSUER  \
FUND                                 Datetime                                   
METLIFE INC                          2024-05-31                 CREDICORP LTD   
                                     2024-05-31         PAGSEGURO DIGITAL LTD   
                                     2024-05-31                        XP INC   
                                     2024-05-31       INTERCORP FINL SVCS INC   
                                     2024-05-31              COPA HOLDINGS SA   
...                                                                       ...   
BANK VONTOBEL AG                     2024-08-31  THERMO FISHER SCIENTIFIC INC   
Vontobel Swiss Financial Advisers AG 2024-08-31  THERMO FISHER SCIENTIFIC INC   
Bank Vontobel Europe AG              2024-08-31  THERMO FISHER SCIENTIFIC INC   
Vontobel Asset Management Ltd        2024-08-31  THERMO FISHER SCIENTIFIC INC   
Vontobel Asset Management S.A.       2024-08-31  THERMO FISHER SCIENTIFIC INC   

                                                TITLEOFCLASS      CUSIP  \
FUND                                 Datetime                             
METLIFE INC                          2024-05-31          COM  G2519Y108   
                                     2024-05-31     COM CL A  G68707101   
                                     2024-05-31         CL A  G98239109   
                                     2024-05-31          SHS  P5626F128   
                                     2024-05-31         CL A  P31076105   
...                                                      ...        ...   
BANK VONTOBEL AG                     2024-08-31          COM  883556102   
Vontobel Swiss Financial Advisers AG 2024-08-31          COM  883556102   
Bank Vontobel Europe AG              2024-08-31          COM  883556102   
Vontobel Asset Management Ltd        2024-08-31          COM  883556102   
Vontobel Asset Management S.A.       2024-08-31          COM  883556102   

                                                   VALUE  SSHPRNAMT  \
FUND                                 Datetime                         
METLIFE INC                          2024-05-31   757896       5055   
                                     2024-05-31   119700       9599   
                                     2024-05-31    48438       1858   
                                     2024-05-31  1336645      60895   
                                     2024-05-31  1680442      15807   
...                                                  ...        ...   
BANK VONTOBEL AG                     2024-08-31   207539        391   
Vontobel Swiss Financial Advisers AG 2024-08-31   207539        391   
Bank Vontobel Europe AG              2024-08-31   207539        391   
Vontobel Asset Management Ltd        2024-08-31   207539        391   
Vontobel Asset Management S.A.       2024-08-31   207539        391   

                                                          aum    aum_date  \
FUND                                 Datetime                               
METLIFE INC                          2024-05-31           NaN         NaN   
                                     2024-05-31           NaN         NaN   
                                     2024-05-31           NaN         NaN   
                                     2024-05-31           NaN         NaN   
                                     2024-05-31           NaN         NaN   
...                                                       ...         ...   
BANK VONTOBEL AG                     2024-08-31           NaN         NaN   
Vontobel Swiss Financial Advisers AG 2024-08-31  8.500000e+09  2023-03-29   
Bank Vontobel Europe AG              2024-08-31           NaN         NaN   
Vontobel Asset Management Ltd        2024-08-31           NaN         NaN   
Vontobel Asset Management S.A.       2024-08-31           NaN         NaN   

                                                    RANK  \
FUND                                 Datetime    

In [18]:
# revome if no shares
df_full = df_full[df_full["SSHPRNAMT"] != 0]

In [19]:
# Number of share to matter
n = 10000 # we are going to ignore holdings of less than 5000 shares
df_full = df_full[df_full["SSHPRNAMT"] > n]

# find unique cusips to download from yfinance

unique_cusips = df_full["CUSIP"].unique()

In [20]:
len(unique_cusips)

18435

# Ticker
Citation: Wharton Research Data Services. "WRDS" wrds.wharton.upenn.edu, accessed 2024-11-29.

In [21]:
from sec_api import MappingApi
mappingApi = MappingApi(api_key="4f82f10de0f412bad35e248c4ef0f35af336fcfde62ccefdd28d3fbab8e04e0f")


In [22]:
test_result = mappingApi.resolve('cusip', 'G2519Y108')

In [23]:
print(test_result[0])


{'name': 'CREDICORP LTD', 'ticker': 'BAP', 'cik': '1001290', 'cusip': 'G2519Y108', 'exchange': 'NYSE', 'isDelisted': False, 'category': 'ADR Common Stock', 'sector': 'Financial Services', 'industry': 'Banks - Regional', 'sic': '6029', 'sicSector': 'Finance Insurance And Real Estate', 'sicIndustry': 'Commercial Banks Nec', 'famaSector': '', 'famaIndustry': 'Banking', 'currency': 'PEN', 'location': 'Bermuda', 'id': '8265d79312a09b467891d613065b5f93'}


In [24]:
date = '2024-08-31'
df_full.loc["METLIFE INC"].sort_values("RANK")

,NAMEOFISSUER,TITLEOFCLASS,CUSIP,VALUE,SSHPRNAMT,aum,aum_date,RANK,TotalHoldingsMarketValue,Percentage,Previous Percentage,ChangeInPercentage
Datetime,,,,,,,,,,,,
2024-05-31,ISHARES TR,IBOXX INV CP ETF,464287242,921469348,8406800,NaN,NaN,1.0,2.184377e+11,4.218455e-01,0.020864,-0.400981
2024-08-31,MICROSOFT CORP,COM,594918104,1040306071,2327567,NaN,NaN,1.0,2.111874e+10,4.925985e+00,0.037682,-4.888303
2024-08-31,APPLE INC,COM,037833100,952286948,4521351,NaN,NaN,2.0,2.111874e+10,4.509203e+00,0.003477,-4.505726
2024-05-31,ISHARES TR,IBOXX INV CP ETF,464287242,893382191,8473700,NaN,NaN,2.0,2.184377e+11,4.089873e-01,0.016460,-0.392528
2024-05-31,MICROSOFT CORPORATION,COM,594918104,861591226,2047897,NaN,NaN,3.0,2.184377e+11,3.944335e-01,0.003604,-0.390830
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-31,ZIOPHARM ONCOLOGY INC,COM,98973P101,13190,12101,NaN,NaN,43759.0,2.184377e+11,6.038336e-06,0.001000,0.000994
2024-05-31,MULTIPLAN CORP,COM,62548M100,10844,13368,NaN,NaN,43850.0,2.184377e+11,4.964345e-06,0.000121,0.000116
2024-05-31,VBI VACCINES INC CDA,COM NEW,91822J103,10050,12431,NaN,NaN,43875.0,2.184377e+11,4.600855e-06,0.000274,0.000270


In [25]:
df_full.loc["METLIFE INC"].sort_values("Percentage",ascending=False)

,NAMEOFISSUER,TITLEOFCLASS,CUSIP,VALUE,SSHPRNAMT,aum,aum_date,RANK,TotalHoldingsMarketValue,Percentage,Previous Percentage,ChangeInPercentage
Datetime,,,,,,,,,,,,
2024-08-31,MICROSOFT CORP,COM,594918104,1040306071,2327567,NaN,NaN,1.0,2.111874e+10,4.925985e+00,0.037682,-4.888303
2024-08-31,APPLE INC,COM,037833100,952286948,4521351,NaN,NaN,2.0,2.111874e+10,4.509203e+00,0.003477,-4.505726
2024-08-31,NVIDIA CORP,COM,67066G104,945075811,7649958,NaN,NaN,3.0,2.111874e+10,4.475057e+00,0.002031,-4.473026
2024-08-31,AMAZON COM INC,COM,023135106,555798595,2876060,NaN,NaN,4.0,2.111874e+10,2.631779e+00,0.001350,-2.630428
2024-08-31,SPDR S&P 500 ETF TR TR UNIT,TR UNIT,78462F103,506156511,966890,NaN,NaN,5.0,2.111874e+10,2.396717e+00,0.000648,-2.396069
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-31,ZIOPHARM ONCOLOGY INC,COM,98973P101,13190,12101,NaN,NaN,43759.0,2.184377e+11,6.038336e-06,0.001000,0.000994
2024-05-31,MULTIPLAN CORP,COM,62548M100,10844,13368,NaN,NaN,43850.0,2.184377e+11,4.964345e-06,0.000121,0.000116
2024-05-31,VBI VACCINES INC CDA,COM NEW,91822J103,10050,12431,NaN,NaN,43875.0,2.184377e+11,4.600855e-06,0.000274,0.000270


In [26]:
df_full.index.value_counts()

FUND                                  Datetime  
PARAMETRIC PORTFOLIO ASSOCIATES LLC   2024-08-31    95145
GOLDMAN SACHS ASSET MANAGEMENT, L.P.  2024-05-31    94380
AYCO CO L P                           2024-05-31    93094
GOLDMAN SACHS & CO. LLC               2024-05-31    93094
GOLDMAN SACHS BANK AG                 2024-05-31    93094
                                                    ...  
TimesSquare Capital Management, LLC   2024-05-31        1
                                      2024-08-31        1
Foundry Group, LLC                    2024-08-31        1
                                      2024-05-31        1
 Burgundy Asset Management Ltd.       2024-08-31        1
Name: count, Length: 6808, dtype: int64